In [30]:
from scipy.sparse.linalg import eigs
import numpy as np
from matplotlib import pyplot as plt

#vals, vecs = eigs(pi.H_matrix, k=5,  which='SM')
#print(vals[0])#

#plt.plot(pi.x, vecs[:,3], 'red')
#plt.plot(pi.x, pi.V, 'b--')

# m is the mass, N is the number of grid points along x, dx is the spacing in x
def finite_difference_T_matrix(m, N, dx):
    ''' check out: http://sites.science.oregonstate.edu/~roundyd/COURSES/ph366/free-particle.html
    '''
    hbar = 1
    T_matrix = np.zeros((3*N,3*N))
    t0 = hbar ** 2 /(2 * m * dx ** 2)
    # need d^2/dx^2 and d^2/dy^2
    for i in range(0,3*N):
        T_matrix[i,i] = 2*t0
        if i<N-1 and i>0:
            T_matrix[i,i+1] = -t0
            T_matrix[i,i-1] = -t0
        elif i<N-1:
            T_matrix[i,i+1] = -t0
        elif i>0:
            T_matrix[i,i-1] = -t0
    return T_matrix
    
#def finite_difference_V_matrix(self):
#    N = self.grid_points
#    for i in range(0,N):
#        self.V_matrix[i,i] = self.V[i]
#    return 1
    
#def finite_difference_H_matrix(self):
#    self.H_matrix = self.T_matrix + self.V_matrix
#    return 1


N = 100
x = np.linspace(0, 2, N)
#y = np.linspace(0, 2, N)

#xy = np.append(x,y)
dx = x[1]

T = finite_difference_T_matrix(1, N, dx)

print(len(T))
E_opt, c_opt = np.linalg.eigh(T)
print(E_opt[0])

Eigs = []
for i in range(1,20):
    Eigs.append(i**2 * np.pi**2/(2*2**2))
    
print(Eigs)

#plt.plot(x, vecs[:N,0], 'red')
#plt.plot(y, vecs[N:,0], 'blue')

300
0.13345757030040406
[1.2337005501361697, 4.934802200544679, 11.103304951225528, 19.739208802178716, 30.842513753404244, 44.41321980490211, 60.45132695667232, 78.95683520871486, 99.92974456102975, 123.37005501361698, 149.27776656647654, 177.65287921960845, 208.49539297301268, 241.80530782668927, 277.5826237806382, 315.82734083485946, 356.5394589893531, 399.718978244119, 445.3658985991573]


In [ ]:
#function[E,psi]=Schroed2D_FEM_f(x,y,V0,Mass,n)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Constants %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

h=6.62606896E-34;              # %% Planck constant [J.s]
hbar=h/(2*pi);
e=1.602176487E-19;            #  %% electron charge [C]
m0=9.10938188E-31;            #  %% electron mass [kg]

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Nx=length(x);
Ny=length(y);

dx=x(2)-x(1);
dy=y(2)-y(1);

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%% Building of the operators %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#%%%%%%%%%%%%%%%%%%%%%%%%%%%% Second derivative X %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% DX2(Ny=5,Nx=4); %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%                                                                                  %
#%  -2   0   0   0   0 | 1   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
#%   0  -2   0   0   0 | 0   1   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
#%   0   0  -2   0   0 | 0   0   1   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
#%   0   0   0  -2   0 | 0   0   0   1   0 | 0   0   0   0   0 | 0   0   0   0   0  %
#%   0   0   0   0  -2 | 0   0   0   0   1 | 0   0   0   0   0 | 0   0   0   0   0  %
#%   -----------------------------------------------------------------------------  %
#%   1   0   0   0   0 |-2   0   0   0   0 | 1   0   0   0   0 | 0   0   0   0   0  %
#%   0   1   0   0   0 | 0  -2   0   0   0 | 0   1   0   0   0 | 0   0   0   0   0  %
#%   0   0   1   0   0 | 0   0  -2   0   0 | 0   0   1   0   0 | 0   0   0   0   0  %
#%   0   0   0   1   0 | 0   0   0  -2   0 | 0   0   0   1   0 | 0   0   0   0   0  %
#%   0   0   0   0   1 | 0   0   0   0  -2 | 0   0   0   0   1 | 0   0   0   0   0  %
#%   -----------------------------------------------------------------------------  %
#%   0   0   0   0   0 | 1   0   0   0   0 |-2   0   0   0   0 | 1   0   0   0   0  %
#%   0   0   0   0   0 | 0   1   0   0   0 | 0  -2   0   0   0 | 0   1   0   0   0  %
#%   0   0   0   0   0 | 0   0   1   0   0 | 0   0  -2   0   0 | 0   0   1   0   0  %
#%   0   0   0   0   0 | 0   0   0   1   0 | 0   0   0  -2   0 | 0   0   0   1   0  %
#%   0   0   0   0   0 | 0   0   0   0   1 | 0   0   0   0  -2 | 0   0   0   0   1  %
#%   -----------------------------------------------------------------------------  %
#%   0   0   0   0   0 | 0   0   0   0   0 | 1   0   0   0   0 |-2   0   0   0   0  %
#%   0   0   0   0   0 | 0   0   0   0   0 | 0   1   0   0   0 | 0  -2   0   0   0  %
#%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   1   0   0 | 0   0  -2   0   0  %
#%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   1   0 | 0   0   0  -2   0  %
#%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   1 | 0   0   0   0  -2  %
#%                                                                                  %
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Axy = ones(1,(Nx-1)*Ny);
DX2 = (-2)*diag(ones(1,Ny*Nx)) + (1)*diag(Axy,-Ny) + (1)*diag(Axy,Ny);

%%%%%%%%%%%%%%%%%%%%%%%%%%% Second derivative Y %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% DY2(Ny=5,Nx=4); %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                                  %
%  -2   1   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   1  -2   1   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   1  -2   1   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   1  -2   1 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   0   1  -2 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   -----------------------------------------------------------------------------  %
%   0   0   0   0   0 |-2   1   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 1  -2   1   0   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   1  -2   1   0 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   1  -2   1 | 0   0   0   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   0   1  -2 | 0   0   0   0   0 | 0   0   0   0   0  %
%   -----------------------------------------------------------------------------  %
%   0   0   0   0   0 | 0   0   0   0   0 |-2   1   0   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 1  -2   1   0   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   1  -2   1   0 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   1  -2   1 | 0   0   0   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   1  -2 | 0   0   0   0   0  %
%   -----------------------------------------------------------------------------  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 |-2   1   0   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 1  -2   1   0   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   1  -2   1   0  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   1  -2   1  %
%   0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   0   0 | 0   0   0   1  -2  %
%                                                                                  %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

AA=ones(1,Nx*Ny);
BB=ones(1,Nx*Ny-1);
BB(Ny:Ny:end)=0;

DY2=(-2)*diag(AA) + (1)*diag(BB,-1) + (1)*diag(BB,1);

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%% Building of the Hamiltonien %%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

H=(-hbar^2/(2*m0*Mass)) * ( DX2/dx^2 + DY2/dy^2 ) +  diag(V0(:)*e) ;

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%% Diagonalisation of the Hamiltonien %%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

H=sparse(H);
[PSI,Energy] = eigs(H,n,'SM');
E = diag(Energy)/e;

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%% Normalization of the Wavefunctions %%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

for i=1:n
    psi_temp=reshape(PSI(:,i),Ny,Nx);
    psi(:,:,i) = psi_temp / sqrt( trapz( y' , trapz(x,abs(psi_temp).^2 ,2) , 1 )  );  % normalisation of the wave function psi
end

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% here is a small patch due to differences between Octave and Matlab
% Matlab order the eigen values while Octave reverse it

if E(1)>E(2)
  psi=psi(:,:,end:-1:1);
  E=E(end:-1:1);
end

end
